
## DESCRIPTION

This notebook fits a series of generalized linear mixed models

***
### SETUP

Load the setup script

In [1]:
from utils.common_setup import *

Load the functions to :

- Fit the models
- Save the models
- Load the models

In [2]:
from utils.functions import fit_models, save_fitted_models, load_fitted_models

Import the cleaned dataset

In [3]:
data_folder = os.path.join(os.getcwd(), '..', 'data')
data = pd.read_csv(os.path.join(data_folder, 'clean_data.csv'))

### IF PackageNotInstalledError: The R package "lme4" is not installed. 


See README for instruction


```
from rpy2.robjects.packages import importr


utils = importr('utils')
utils.chooseCRANmirror(ind=12)
utils.install_packages('lme4')
```

### FIT GLMM

### Define model formulas

There are 33 models starting from model 0

In [21]:
model_formulas = [
    
    # Null model ------------------------------------------------

    # model0
    "death_next_season ~ 1 + (1|region)",

    # Models with aqhi average and region -----------------------

    # model1
    "death_next_season ~ aqhi_average + ndvi_average + (aqhi_average|region)",

    # model2
    "death_next_season ~ aqhi_average * ndvi_average + (aqhi_average|region)",

    # model3
    "death_next_season ~ aqhi_average + ndvi_average + tavg_average + prcp_average + wspd_average + (aqhi_average|region)",

    # model4
    "death_next_season ~ aqhi_average * wspd_average + ndvi_average + prcp_average + tavg_average + (aqhi_average|region)",

    # model5
    "death_next_season ~ aqhi_average * ndvi_average + wspd_average + (ndvi_average|region)",

    # model6
    "death_next_season ~ aqhi_average * wspd_average * tavg_average + (aqhi_average|region)",

    # model7
    "death_next_season ~ aqhi_average * wspd_average * ndvi_average + (ndvi_average|region)",

    # Models with aqhi average and operation id -----------------

    # model8
    "death_next_season ~ aqhi_average * ndvi_average + (1|operation_id)",

    # model9
    "death_next_season ~ aqhi_average * ndvi_average + (aqhi_average|operation_id)",

    # model10
    "death_next_season ~ aqhi_average + ndvi_average + wspd_average + (1|operation_id)",

    # model11
    "death_next_season ~ aqhi_average + ndvi_average + wspd_average + (aqhi_average|operation_id)",

    # model12
    "death_next_season ~ aqhi_average * wspd_average + ndvi_average + (aqhi_average|operation_id)",

    # model13
    "death_next_season ~ aqhi_average * ndvi_average + wspd_average + (1|operation_id)",

    # model14
    "death_next_season ~ aqhi_average * ndvi_average * wspd_average + (1|operation_id)",

    # model15
    "death_next_season ~ aqhi_average * ndvi_average * wspd_average + (aqhi_average|operation_id)",

    # model16
    "death_next_season ~ aqhi_average + ndvi_average + wspd_average + tavg_average + prcp_average + (aqhi_average|operation_id)",

    # model17
    "death_next_season ~ aqhi_average * ndvi_average + tavg_average * prcp_average + wspd_average + (aqhi_average|operation_id)",

    # Models with aqhi skew and region --------------------------
    
    # model18
    "death_next_season ~ aqhi_skew + ndvi_skew + (aqhi_skew|region)",

    # model19
    "death_next_season ~ aqhi_skew * ndvi_skew + (aqhi_skew|region)",

    # model20
    "death_next_season ~ aqhi_skew * ndvi_average + wspd_average + (aqhi_skew|region)",

    # model21
    "death_next_season ~ aqhi_skew * ndvi_average * wspd_average + (aqhi_skew|region)",

    # Models with aqhi max ---------------------------

    # model22
    "death_next_season ~ aqhi_max * ndvi_average + wspd_average + (ndvi_average|region)",

    # model23
    "death_next_season ~ aqhi_max * ndvi_average * wspd_average + (ndvi_average|region)",

    # model24
    "death_next_season ~ aqhi_max * ndvi_average + wspd_average + (aqhi_average|operation_id)",

    # model25
    "death_next_season ~ aqhi_max * ndvi_average * wspd_average + (aqhi_average|operation_id)",

    # Models with ndvi skew --------------------------

    # model26
    "death_next_season ~ aqhi_average * ndvi_skew + wspd_average + (aqhi_average|region)",

    # model27
    "death_next_season ~ aqhi_average * ndvi_skew * wspd_average + (aqhi_average|region)",

    # model28
    "death_next_season ~ aqhi_average * ndvi_skew * wspd_average + (aqhi_average|operation_id)",

    # Models with O3 and operation id ---------------------------

    # model29
    "death_next_season ~ ndvi_average + o3_average + (o3_average|operation_id)",

    # model30
    "death_next_season ~ ndvi_average * o3_average + (o3_average|operation_id)",

    # model31
    "death_next_season ~ ndvi_average * o3_average + wspd_average + (o3_average|operation_id)",

    # model32
    "death_next_season ~ ndvi_average * o3_average * wspd_average + (o3_average|operation_id)"

]

### Standardize all variables

In [5]:
from sklearn.preprocessing import StandardScaler

# Variables
variables_to_scale = [
    'aqhi_average', 'ndvi_average', 'wspd_average', 
    'tavg_average', 'prcp_average', 'o3_average',
    'aqhi_skew', 'aqhi_max', 'ndvi_skew'
]

# Initialize the scaler
scaler = StandardScaler()

# Overwrite original variables names with scaled variables
data[variables_to_scale] = scaler.fit_transform(data[variables_to_scale])

In [15]:
print(data[variables_to_scale].describe())

       aqhi_average  ndvi_average  wspd_average  tavg_average  prcp_average  \
count  1.126260e+05  1.126260e+05  1.126260e+05  1.126260e+05  1.126260e+05   
mean  -2.099592e-16 -1.736201e-16 -3.391648e-16 -3.109011e-15  4.037676e-17   
std    1.000004e+00  1.000004e+00  1.000004e+00  1.000004e+00  1.000004e+00   
min   -4.424029e+00 -8.433916e+00 -3.346216e+01 -1.436220e+01 -5.548384e+00   
25%   -7.151574e-01 -4.662709e-01 -2.370080e-01 -6.814093e-01 -9.297429e-01   
50%   -2.315185e-01 -1.513841e-01  1.031090e-01 -1.269333e-01  4.509853e-01   
75%    9.832993e-01  3.347263e-01  4.154009e-01  6.577897e-01  7.345448e-01   
max    2.308546e+00  1.472707e+01  2.951177e+00  3.108767e+00  2.473830e+00   

         o3_average     aqhi_skew      aqhi_max     ndvi_skew  
count  1.126260e+05  1.121070e+05  1.126260e+05  1.121570e+05  
mean   4.602951e-16 -5.678916e-17  1.049796e-16  3.745400e-16  
std    1.000004e+00  1.000004e+00  1.000004e+00  1.000004e+00  
min   -3.246371e+01 -4.412670e+0

Inspect how many models

In [19]:
len(model_formulas)

32

In [7]:
data.columns

Index(['hid', 'creation_date', 'operation_id', 'death_date', 'season',
       'season_start_month', 'season_start_day', 'death_next_season',
       'hive_age_next_season', 'aqhi_average', 'prcp_average', 'wspd_average',
       'ndvi_average', 'tavg_average', 'o3_average', 'aqhi_skew', 'prcp_skew',
       'wspd_skew', 'ndvi_skew', 'tavg_skew', 'o3_skew', 'aqhi_max',
       'prcp_max', 'wspd_max', 'ndvi_max', 'tavg_max', 'o3_max', 'region',
       'wspd_average_og', 'tavg_average_og', 'prcp_average_og',
       'aqhi_average_og'],
      dtype='object')

### Fit the models and save (**run once**)

Fitting all models takes **approximately 1.2 hours.**

Run once and save in ```aqi/outputs``` folder so you can reimport in session.

File is called ```fitted_GLMMs.pkl```

In [22]:
fitted_models, model_summaries = fit_models(
    data=data,
    formulas=model_formulas
)

save_fitted_models(
    fitted_models=fitted_models,
    filename="fitted_GLMMs.pkl"
)

save_fitted_models(
    fitted_models=model_summaries,
    filename="summaries_GLMMs.pkl"
)

Fitting Models:   0%|          | 0/33 [00:00<?, ?it/s]

Fitting model0 with formula: death_next_season ~ 1 + (1|region)


Fitting Models:   3%|▎         | 1/33 [00:14<07:37, 14.30s/it]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~1+(1|region)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'region': 9.0}

Log-likelihood: -71157.617 	 AIC: 142319.235

Random effects:

               Name    Var    Std
region  (Intercept)  0.979  0.989

No random effect correlations specified

Fixed effects:

Fitting model1 with formula: death_next_season ~ aqhi_average + ndvi_average + (aqhi_average|region)


Fitting Models:   6%|▌         | 2/33 [00:49<13:38, 26.39s/it]

...

Fitting Models:  97%|█████████▋| 32/33 [1:13:01<01:22, 82.18s/it]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~ndvi_average*o3_average+wspd_average+(o3_average|operation_id)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'operation_id': 17.0}

Log-likelihood: -64613.069 	 AIC: 129242.139

Random effects:

                     Name    Var    Std
operation_id  (Intercept)  3.479  1.865
operation_id   o3_average  3.880  1.970

                      IV1         IV2   Corr
operation_id  (Intercept)  o3_average  0.688

Fixed effects:

Fitting model32 with formula: death_next_season ~ ndvi_average * o3_average * wspd_average + (o3_average|operation_id)


Fitting Models: 100%|██████████| 33/33 [1:14:40<00:00, 135.77s/it]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~ndvi_average*o3_average*wspd_average+(o3_average|operation_id)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'operation_id': 17.0}

Log-likelihood: -64579.417 	 AIC: 129180.833

Random effects:

                     Name    Var    Std
operation_id  (Intercept)  3.432  1.853
operation_id   o3_average  3.920  1.980

                      IV1         IV2   Corr
operation_id  (Intercept)  o3_average  0.687

Fixed effects:



Models saved to /Users/maximeff-nectar/Documents/ledge-nectar/aqi/outputs/fitted_GLMMs.pkl.
Models saved to /Users/maximeff-nectar/Documents/ledge-nectar/aqi/outputs/summaries_GLMMs.pkl.


### Load models in session

In [10]:
fitted_models = load_fitted_models("fitted_GLMMs.pkl")

Models loaded from /Users/maximeff-nectar/Documents/ledge-nectar/aqi/outputs/fitted_GLMMs.pkl.


In [23]:
# Initialize variables
best_aic = float('inf')
best_model = None
best_model_name = None

# Loop over the fitted models dict
for model_name, model in fitted_models.items():
    
    # Check if the model has warnings
    if len(model.warnings) == 0:
        # Get the AIC for the current model
        current_aic = model.AIC
        
        # Compare with the best AIC found so far
        if current_aic < best_aic:
            print(f"Current best model is {model_name} with an AIC of {current_aic}")
            best_aic = current_aic
            best_model = model
            best_model_name = model_name
    else:
        print(f"Model {model_name} did not converge due to warnings: {model.warnings}")

Current best model is model0 with an AIC of 142319.23490165427
Current best model is model1 with an AIC of 138979.96571818696
Current best model is model2 with an AIC of 138707.4923684611
Current best model is model3 with an AIC of 136911.5749373824
Current best model is model4 with an AIC of 136899.1582601408
Current best model is model6 with an AIC of 136699.7219109801
Current best model is model8 with an AIC of 132882.71574743759
Current best model is model9 with an AIC of 131480.50204320857
Current best model is model15 with an AIC of 131428.72718470442
Current best model is model16 with an AIC of 129574.36442449133
Current best model is model17 with an AIC of 129536.02753766163
Current best model is model29 with an AIC of 129405.29649029924
Current best model is model30 with an AIC of 129319.57520043233
Current best model is model31 with an AIC of 129242.13857908593
Current best model is model32 with an AIC of 129180.83310119298


In [24]:
print(best_model.fit())

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: death_next_season~ndvi_average*o3_average*wspd_average+(o3_average|operation_id)

Family: binomial	 Inference: parametric

Number of observations: 112626	 Groups: {'operation_id': 17.0}

Log-likelihood: -64579.417 	 AIC: 129180.833

Random effects:

                     Name    Var    Std
operation_id  (Intercept)  3.432  1.853
operation_id   o3_average  3.920  1.980

                      IV1         IV2   Corr
operation_id  (Intercept)  o3_average  0.687

Fixed effects:

                                      Estimate  2.5_ci  97.5_ci     SE     OR  \
(Intercept)                             -0.149  -1.033    0.735  0.451  0.862   
ndvi_average                            -0.403  -0.422   -0.384  0.010  0.668   
o3_average                               1.229   0.291    2.166  0.478  3.417   
wspd_average                            -0.078  -0.093   -0.064  0.007  0.925   
ndvi_average:o3_average                 -0.098  -0

Save the best model output

In [25]:
project_root = os.path.dirname(os.getcwd())
output_dir = os.path.join(project_root, 'outputs')    
filepath = os.path.join(output_dir, "GLMM_best_model.joblib")

save_model(best_model, filepath = filepath)